In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the MS-COCO dataset from TensorFlow Datasets
train_data, info = tfds.load('coco/2017', split='train[:100]', with_info=True)
test_data = tfds.load('coco/2017', split='test[:50]', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [1]:
# Check the number of training and testing images
num_train_images = info.splits['train'].num_examples
num_test_images = info.splits['test'].num_examples
print("Number of training images:", num_train_images)
print("Number of testing images:", num_test_images)

NameError: name 'info' is not defined

In [ ]:
def plot_sample_images(dataset, num_images=5):
    plt.figure(figsize=(15, 10))
    for i, sample in enumerate(dataset.take(num_images)):
        image = sample['image']
        plt.subplot(1, num_images, i + 1)
        plt.imshow(image)
        plt.axis('off')
    plt.show()

plot_sample_images(train_data)

In [ ]:
augmentor = ImageDataGenerator(
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2]
)

# Apply augmentation and visualize a few images
def plot_augmented_images(augmentor, dataset, num_images=5):
    plt.figure(figsize=(15, 10))
    for i, sample in enumerate(dataset.take(num_images)):
        image = sample['image'].numpy()
        augmented_image = augmentor.random_transform(image)
        plt.subplot(1, num_images, i + 1)
        plt.imshow(augmented_image / 255.0)
        plt.axis('off')
    plt.show()

plot_augmented_images(augmentor, train_data)

In [ ]:
def normalize(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, label

train_data = train_data.map(lambda x: normalize(x['image'], x['label']), num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.map(lambda x: normalize(x['image'], x['label']), num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(80, activation='softmax')  # Assuming 80 classes as in MS-COCO
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_cnn = cnn_model.fit(train_data.batch(32), epochs=10, validation_data=test_data.batch(32))

# Display training and testing accuracy
train_acc_cnn = history_cnn.history['accuracy'][-1]
test_acc_cnn = history_cnn.history['val_accuracy'][-1]
print("Training Accuracy (CNN):", train_acc_cnn)
print("Testing Accuracy (CNN):", test_acc_cnn)

In [ ]:
# Use TensorFlow’s Model Zoo for pre-trained Faster R-CNN on COCO
from object_detection.models import faster_rcnn_resnet50_v1_fpn_keras

faster_rcnn_model = faster_rcnn_resnet50_v1_fpn_keras()
faster_rcnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train Faster R-CNN model
history_faster_rcnn = faster_rcnn_model.fit(train_data.batch(32), epochs=10, validation_data=test_data.batch(32))

# Display training and testing accuracy
train_acc_faster_rcnn = history_faster_rcnn.history['accuracy'][-1]
test_acc_faster_rcnn = history_faster_rcnn.history['val_accuracy'][-1]
print("Training Accuracy (Faster R-CNN):", train_acc_faster_rcnn)
print("Testing Accuracy (Faster R-CNN):", test_acc_faster_rcnn)

In [ ]:
print("CNN - Training Accuracy (After Augmentation):", train_acc_cnn)
print("CNN - Testing Accuracy (After Augmentation):", test_acc_cnn)
print("Faster R-CNN - Training Accuracy (After Augmentation):", train_acc_faster_rcnn)
print("Faster R-CNN - Testing Accuracy (After Augmentation):", test_acc_faster_rcnn)

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt

# Function to plot training and validation accuracy
def plot_accuracy(history, title):
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras import layers

# Load the MS-COCO dataset
dataset, info = tfds.load('coco/2017', with_info=True, as_supervised=True)

# Split the dataset into training and validation (testing) sets
train_data = dataset['train']
val_data = dataset['validation']

# Check the number of training and testing images
train_size = info.splits['train'].num_examples
val_size = info.splits['validation'].num_examples

print(f'Training Images: {train_size}')
print(f'Validation Images: {val_size}')

def plot_images(dataset, num_images=5):
    plt.figure(figsize=(10, 10))
    for i, (image, label) in enumerate(dataset.take(num_images)):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(image)
        plt.axis('off')
    plt.show()

plot_images(train_data)

def augment_image(image):
    image = tf.image.random_flip_left_right(image)  # Random horizontal flip
    image = tf.image.random_flip_up_down(image)    # Random vertical flip
    image = tf.image.random_contrast(image, lower=0.2, upper=0.5)  # Random contrast
    image = tf.image.random_rotation(image, 0.2)   # Random rotation
    return image

# Apply augmentation on the dataset
train_data = train_data.map(lambda image, label: (augment_image(image), label))
val_data = val_data.map(lambda image, label: (augment_image(image), label))

normalization_layer = layers.Rescaling(1.0 / 255)
train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))

# Build a CNN model
model = tf.keras.Sequential([
    layers.InputLayer(input_shape=(128, 128, 3)),  # Adjust size to 128x128 for simplicity
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data.batch(32), validation_data=val_data.batch(32), epochs=10)


# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras import Model

# Load the COCO dataset
data_dir = 'coco_dataset'

# Get the list of image files
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_files, [0] * len(image_files), test_size=0.1, random_state=42)

print(f"Number of training images: {len(X_train)}")
print(f"Number of testing images: {len(X_test)}")

# Plot some sample images
plt.figure(figsize=(12, 6))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(plt.imread(X_train[i]))
    plt.axis('off')
plt.show()

# Perform image augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    contrast_stretch_range=(0.8, 1.2)
)

# Fit and generate augmented training data
train_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Get the updated number of training and testing images
X_train, y_train = next(train_generator)
print(f"Number of training images after augmentation: {len(X_train)}")
print(f"Number of testing images: {len(X_test)}")

# Normalize the training data
X_train = X_train / 255.0
X_test = np.array([plt.imread(f) for f in X_test]) / 255.0

# Build a convolutional neural network
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(80, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Show the training and testing accuracy
print(f"Training accuracy: {history.history['accuracy'][-1]:.2f}")
print(f"Testing accuracy: {history.history['val_accuracy'][-1]:.2f}")

# Build a Faster R-CNN
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in model.layers:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(80, activation='softmax')(x)

model = Model(inputs=model.input, outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Show the training and testing accuracy
print(f"Training accuracy: {history.history['accuracy'][-1]:.2f}")
print(f"Testing accuracy: {history.history['val_accuracy'][-1]:.2f}")